In [1]:
import open3d as o3d
import os
import numpy as np
from tqdm import tqdm
import math

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


### Dataset Creation

In [2]:
# combine clouds
data_path = "/mnt/f/datasets/export/export/"
max_points = 4096

In [3]:
classes = os.listdir(data_path)
print(classes)

['Angles', 'Channels', 'CircularHollow', 'Conduit', 'Elbows', 'Flanges', 'Ibeams', 'Other', 'Pipes', 'Valves']


In [4]:
all_classes = []
element_count = 0
error_count = 0
for i, cl in enumerate(classes):
    all_elements = []
    class_path = data_path + cl
    elements = os.listdir(class_path)
    for j, el in tqdm(enumerate(elements)):
        try:
            element = np.loadtxt(class_path + '/' + el)

            # downsample
            if (len(element) > 0 and element.ndim == 2 and element.shape[1] == 4):
                if len(element)  > max_points:
                    #idx = np.random.randint(element.shape[0], size=max_points)
                    #element = element[idx :]
                    element = np.random.permutation(element)[:max_points]

                element = np.delete(element, 3, axis=1) # remove point index
                element = np.insert(element, 3, values=[element_count], axis=1) # add element index
                #print(element.shape)
                element_count += 1
                all_elements.append(element)
        except Exception as E:
            error_count += 1
            
            
    all_elements = np.vstack(all_elements)
    all_elements = np.insert(all_elements, 4, values=[i], axis=1) # add class index
    all_classes.append(all_elements)
    
all_classes = np.concatenate(all_classes)
print(all_classes.shape)
print ("errors: ", error_count)
        
        #print(points[0])

211it [06:34,  1.87s/it]
94it [05:03,  3.23s/it]
113it [1:19:18, 42.11s/it]
1361it [29:40,  1.31s/it]
707it [11:34,  1.02it/s]
204it [02:53,  1.18it/s]
116it [26:34, 13.75s/it]
553it [1:48:18, 11.75s/it] 
835it [56:57,  4.09s/it]
192it [06:52,  2.15s/it]


(17490596, 5)
errors:  0


In [6]:
pcd = o3d.t.geometry.PointCloud()
points = all_classes[:,0:3]
el_index  = [[i] for i in all_classes[:,3]]
cl_index  = [[i] for i in all_classes[:,4]]


pcd.point["positions"] = o3d.core.Tensor(points)
pcd.point["elements"] = o3d.core.Tensor(el_index)
pcd.point["classes"] = o3d.core.Tensor(cl_index)

o3d.t.io.write_point_cloud("water2.pcd", pcd)


True

### Pipe parameter detection

#### Generation of synethetic IFC element dataset

In [ ]:
# elbow - modelled as an IfcRevolvedAreaSolid 

element_class = 'elbow'
radius_range = [0.01, 0.9]
angle_range = [0, math.pi]


In [ ]:
# pipe - modelled as an IfcExtrudedAreaSolid

element_class = 'pipe'
radius_range = [0.01, 0.9]
length_range = [0.01, 10.0]
extrusion_direction_range = [[-1.0, 1.0], [-1.0, 1.0], [0.0, 1.0]]
point_range = [-100.0,100.0]

